In [2]:
import os
from uuid import uuid4
from operator import itemgetter
from typing import Optional, List, Tuple

from langchain.chat_models import ChatOpenAI
from langchain.memory.chat_message_histories import ZepChatMessageHistory
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.prompts.prompt import PromptTemplate
from langchain.schema import AIMessage
from langchain.schema.chat_history import BaseChatMessageHistory
from langchain.schema.runnable.history import RunnableWithMessageHistory
from langchain.vectorstores.zep import CollectionConfig, ZepVectorStore

from langchain_core.runnables import (
    ConfigurableField,
    RunnableBranch,
    RunnableLambda,
    RunnableParallel,
    RunnablePassthrough,
)
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables.utils import ConfigurableFieldSingleOption


In [4]:
ZEP_API_URL = os.environ.get("ZEP_API_URL", "http://localhost:8080")
ZEP_API_KEY = os.environ.get("ZEP_API_KEY", None)
ZEP_COLLECTION_NAME = os.environ.get("ZEP_COLLECTION", "langchaintest")

In [6]:
session_id = str(uuid4())
session_id

'f93fd0ea-3735-4a02-a0d4-c48f65295111'

In [4]:
collection_config = CollectionConfig(
    name=ZEP_COLLECTION_NAME,
    description="Zep collection for LangChain",
    metadata={},
    embedding_dimensions=1536,
    is_auto_embedded=True,
)

In [5]:
# vectorstore = ZepVectorStore(
#     collection_name=ZEP_COLLECTION_NAME,
#     config=collection_config,
#     api_url=ZEP_API_URL,
#     api_key=ZEP_API_KEY,
#     embedding=None,
# )

In [6]:
# Zep offers native, hardware-accelerated MMR. Enabling this will improve
# the diversity of results, but may also reduce relevance. You can tune
# the lambda parameter to control the tradeoff between relevance and diversity.
# Enabling is a good default.
# retriever = vectorstore.as_retriever().configurable_fields(
#     search_type=ConfigurableFieldSingleOption(
#         id="search_type",
#         options={"Similarity": "similarity", "Similarity with MMR Reranking": "mmr"},
#         default="mmr",
#         name="Search Type",
#         description="Type of search to perform: 'similarity' or 'mmr'",
#     ),
#     search_kwargs=ConfigurableField(
#         id="search_kwargs",
#         name="Search kwargs",
#         description=(
#             "Specify 'k' for number of results to return and 'lambda_mult' for tuning"
#             " MMR relevance vs diversity."
#         ),
#     ),
# )

In [7]:
# RAG answer synthesis prompt
template = """Answer the question to the best of your knowledge
If the question cannot be answered using the information provided answer
with "I don't know:
"""
# <context>
# {context}
# </context>

QA_PROMPT = ChatPromptTemplate.from_messages(
    [
        ("system", template),
        MessagesPlaceholder(variable_name="chat_history"),
        ("user", "{question}"),
    ]
)


In [8]:
# Conversational Retrieval Chain
# DEFAULT_DOCUMENT_PROMPT = PromptTemplate.from_template(template="{page_content}")

In [9]:
# def _combine_documents(
#     docs: List[Document],
#     document_prompt: PromptTemplate = DEFAULT_DOCUMENT_PROMPT,
#     document_separator: str = "\n\n",
# ):
#     doc_strings = [format_document(doc, document_prompt) for doc in docs]
#     return document_separator.join(doc_strings)

In [12]:
import pprint
session_id = ZepChatMessageHistory(session_id="{session_id}", url=ZEP_API_URL)
session_id

In [10]:
# _inputs = RunnableParallel(
    # {
        # "question": "question"
        # "context": retriever | _combine_documents,
#     }
# )

# chain = _inputs | QA_PROMPT | ChatOpenAI() | StrOutputParser()

In [11]:
chain =  QA_PROMPT | ChatOpenAI() | StrOutputParser()

In [12]:
chain_with_history = RunnableWithMessageHistory(
    chain,
    lambda session_id: ZepChatMessageHistory(session_id, url=ZEP_API_URL),
    input_messages_key="question",
    history_messages_key="chat_history",
)

In [13]:
chain_with_history 

RunnableWithMessageHistory(bound=RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  chat_history: RunnableBinding(bound=RunnableLambda(...), config={'run_name': 'load_history'})
}), config={'run_name': 'insert_history'})
| RunnableBinding(bound=ChatPromptTemplate(input_variables=['chat_history', 'question'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='Answer the question to the best of your knowledge\nIf the question cannot be answered using the information provided answer\nwith "I don\'t know:\n')), MessagesPlaceholder(variable_name='chat_history'), HumanMessagePromptTemplate(prompt=PromptTemplate(input_vari

In [14]:
chain_with_history.invoke(
    {"question":"Who is a firmware engineer"},
    config={"configurable": {"session_id": session_id}},
)

AIMessage(content="A firmware engineer is a professional who is responsible for designing, developing, and maintaining firmware for electronic devices. Firmware is a type of software that provides low-level control for hardware components and is typically embedded within a device's hardware. Firmware engineers work closely with hardware engineers to ensure the firmware is optimized for the specific device and its functionalities. They may also be involved in debugging and troubleshooting firmware-related issues.")

In [15]:
chain_with_history.invoke(
    {"question":"How much do they earn"},
    config={"configurable": {"session_id": session_id}},
)

AIMessage(content='The salary of a firmware engineer can vary depending on factors such as location, level of experience, industry, and company size. In the United States, the average annual salary for a firmware engineer is around $95,000 to $130,000, according to data from Glassdoor and PayScale. However, it is important to note that these figures are just averages and individual salaries may be higher or lower based on various factors.')

In [16]:
chain_with_history.invoke(
    {"question":"what are the basic requirements for a beginner"},
    config={"configurable": {"session_id": session_id}},
)

AIMessage(content="The basic requirements for a beginner to become a firmware engineer may include:\n\n1. Education: A bachelor's degree in electrical engineering, computer engineering, or a related field is typically required. Some employers may also accept candidates with an associate's degree or relevant certifications.\n\n2. Programming Skills: Proficiency in programming languages such as C and C++ is essential for firmware development. Familiarity with assembly language and embedded systems programming is also beneficial.\n\n3. Knowledge of Electronics: Understanding of basic electronics principles and digital logic is important for working with hardware components and designing firmware that interacts with them.\n\n4. Problem-Solving Skills: Firmware engineers often need to analyze and debug complex issues, so strong problem-solving and troubleshooting abilities are crucial.\n\n5. Communication Skills: The ability to effectively communicate and collaborate with cross-functional t

In [ ]:
# chain_with_history.messages